In [183]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
from pymc.math import concatenate
import aesara.tensor as at
from pymc.math import concatenate
from pymc.math import sum as pm_sum
from itertools import permutations, combinations

In [270]:
combos = list(combinations(list(range(4)), 2))
names = [f"α{i}-α{j}" for i, j in combos]
for n, (i, j) in zip(names, combos):
    print(n)
    print(i,j)

α0-α1
0 1
α0-α2
0 2
α0-α3
0 3
α1-α2
1 2
α1-α3
1 3
α2-α3
2 3


In [271]:
yellow = np.array([45, 59, 48, 46, 38, 47])
white = np.array([21, 12, 14, 17, 13, 17])
green = np.array([37, 32, 15, 25, 39, 41])
blue = np.array([16, 11, 20, 21, 14, 7])
# create a (4, 6)-shaped matrix
bugs = np.stack([yellow, white, green, blue], axis=0)
combos = list(combinations(list(range(4)), 2))
names = [f"α{i}-α{j}" for i, j in combos]
with pm.Model():
    mu0 = pm.Normal("μ0", mu=0, tau=0.0001)
    tau = pm.Gamma("τ", 0.001, 0.001)

    a2_4 = pm.Normal("α2_4", mu=0, tau=0.0001, shape=(3, 6))
    a1 = pm.Deterministic("α1", at.reshape(-pm_sum(a2_4, axis=0), (1, 6), ndim=2))
    alpha = pm.Deterministic("α", pm_concat([a1, a2_4], axis=0))

    mu = mu0 + alpha

    pm.Normal("lik", mu=mu, tau=tau, observed=bugs)
    
    for name, (i, j) in zip(names, combos):
        pm.Deterministic(name, alpha[i] - alpha[j])

    trace = pm.sample(5000)

alpha_results = trace.posterior.α.mean(axis=(3))
az.summary(trace)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [μ0, τ, α2_4]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 68 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
μ0,27.279,1.018,25.408,29.261,0.006,0.004,30926.0,10815.0,1.0
"α2_4[0, 0]",-8.727,4.338,-16.620,-0.373,0.026,0.026,28268.0,10482.0,1.0
"α2_4[0, 1]",-16.450,4.358,-24.615,-8.488,0.029,0.022,23875.0,9421.0,1.0
"α2_4[0, 2]",-10.227,4.244,-18.250,-2.326,0.026,0.022,28007.0,11050.0,1.0
"α2_4[0, 3]",-10.241,4.277,-18.268,-2.319,0.025,0.024,27528.0,11810.0,1.0
...,...,...,...,...,...,...,...,...,...
α2-α3[1],20.866,7.061,8.223,34.238,0.045,0.039,24773.0,10597.0,1.0
α2-α3[2],-4.931,7.078,-17.472,8.931,0.044,0.066,26567.0,9487.0,1.0
α2-α3[3],4.005,7.044,-9.015,17.037,0.047,0.067,23647.0,9944.0,1.0
α2-α3[4],24.999,7.020,11.190,37.754,0.046,0.039,23789.0,9401.0,1.0


In [312]:
    for name, (i, j) in zip(names, combos):
        pm.Deterministic(name, alpha[i] - alpha[j])

<xarray.DataArray 'α' (chain: 4, draw: 5000, α_dim_0: 4)>
array([[[ 19.47614762, -11.08295242,   3.44937746, -11.84257266],
        [ 19.47518555, -12.22020396,   5.30302728, -12.55800887],
        [ 20.60578741, -11.97662415,   4.41579129, -13.04495455],
        ...,
        [ 14.59271693,  -7.4603305 ,   3.30068688, -10.4330733 ],
        [ 16.78978897, -12.52934417,   7.19541153, -11.45585633],
        [ 18.73416147, -14.83789418,   6.65578094, -10.55204823]],

       [[ 19.81375587, -10.71897995,   4.69795939, -13.79273531],
        [ 19.54818638, -12.64965239,   6.51049792, -13.40903191],
        [ 19.30219945, -13.53833338,   7.32598173, -13.0898478 ],
        ...,
        [ 20.56974164, -11.58185903,   3.81003228, -12.79791489],
        [ 19.0771394 , -11.55809466,   5.2469933 , -12.76603803],
        [ 20.07320517,  -9.45282606,   4.59806966, -15.21844877]],

       [[ 21.80079764, -15.49894382,   5.51554216, -11.81739598],
        [ 20.48643116,  -9.43659613,   2.86029939, -13.91013442],
        [ 18.78803558,  -9.21119324,   3.85794744, -13.43478978],
        ...,
        [ 20.01697083, -11.91255599,   5.27923336, -13.38364819],
        [ 17.9882214 , -10.18096734,   3.44178098, -11.24903503],
        [ 19.15730032, -13.17452878,   6.60762003, -12.59039157]],

       [[ 17.98964805, -12.07116261,   5.5926634 , -11.51114885],
        [ 21.38071826, -12.17536543,   1.44121189, -10.64656472],
        [ 19.58988949, -12.004993  ,   7.1877244 , -14.77262089],
        ...,
        [ 18.60040731, -12.1681312 ,   3.81884935, -10.25112546],
        [ 18.23280858, -10.68065068,   5.52150715, -13.07366505],
        [ 18.24550825, -11.21732065,   4.29881075, -11.32699835]]])
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 4993 4994 4995 4996 4997 4998 4999
  * α_dim_0  (α_dim_0) int64 0 1 2 3

In [292]:
az.summary(trace, var_names=["~μ0", "~α2_4", "~τ", "~α1"], kind="stats")

,mean,sd,hdi_3%,hdi_97%
"α[0, 0]",15.228,4.348,6.656,23.367
"α[0, 1]",30.476,4.350,22.085,38.548
"α[0, 2]",23.699,4.305,15.400,31.720
"α[0, 3]",18.694,4.349,10.497,26.644
"α[0, 4]",11.974,4.303,4.369,20.343
"α[0, 5]",18.968,4.384,10.670,26.683
"α[1, 0]",-8.727,4.338,-16.620,-0.373
"α[1, 1]",-16.450,4.358,-24.615,-8.488
"α[1, 2]",-10.227,4.244,-18.250,-2.326
"α[1, 3]",-10.241,4.277,-18.268,-2.319


<xarray.Dataset>
Dimensions:      (chain: 4, draw: 5000, α2_4_dim_0: 3, α2_4_dim_1: 6,
                  α1_dim_0: 1, α1_dim_1: 6, α_dim_0: 4, α_dim_1: 6,
                  α0-α1_dim_0: 6, α0-α2_dim_0: 6, α0-α3_dim_0: 6,
                  α1-α2_dim_0: 6, α1-α3_dim_0: 6, α2-α3_dim_0: 6)
Coordinates: (12/14)
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 ... 4994 4995 4996 4997 4998 4999
  * α2_4_dim_0   (α2_4_dim_0) int64 0 1 2
  * α2_4_dim_1   (α2_4_dim_1) int64 0 1 2 3 4 5
  * α1_dim_0     (α1_dim_0) int64 0
  * α1_dim_1     (α1_dim_1) int64 0 1 2 3 4 5
    ...           ...
  * α0-α1_dim_0  (α0-α1_dim_0) int64 0 1 2 3 4 5
  * α0-α2_dim_0  (α0-α2_dim_0) int64 0 1 2 3 4 5
  * α0-α3_dim_0  (α0-α3_dim_0) int64 0 1 2 3 4 5
  * α1-α2_dim_0  (α1-α2_dim_0) int64 0 1 2 3 4 5
  * α1-α3_dim_0  (α1-α3_dim_0) int64 0 1 2 3 4 5
  * α2-α3_dim_0  (α2-α3_dim_0) int64 0 1 2 3 4 5
Data variables:
    μ0           (chain, draw) float64 28.22 26.14 25.79 ... 26.96 28.27 27.22
    α2_4         (chain, draw, α2_4_dim_0, α2_4_dim_1) float64 -6.699 ... -16.02
    τ            (chain, draw) float64 0.04068 0.03447 ... 0.06009 0.03769
    α1           (chain, draw, α1_dim_0, α1_dim_1) float64 15.23 27.81 ... 18.81
    α            (chain, draw, α_dim_0, α_dim_1) float64 15.23 27.81 ... -16.02
    α0-α1        (chain, draw, α0-α1_dim_0) float64 21.93 43.08 ... 17.92 32.42
    α0-α2        (chain, draw, α0-α2_dim_0) float64 6.014 28.29 ... -7.278 7.983
    α0-α3        (chain, draw, α0-α3_dim_0) float64 32.99 39.89 ... 17.67 34.82
    α1-α2        (chain, draw, α1-α2_dim_0) float64 -15.92 -14.8 ... -24.43
    α1-α3        (chain, draw, α1-α3_dim_0) float64 11.06 -3.194 ... 2.409
    α2-α3        (chain, draw, α2-α3_dim_0) float64 26.97 11.6 ... 24.94 26.84
Attributes:
    created_at:                 2022-05-03T03:00:17.422358
    arviz_version:              0.11.4
    inference_library:          pymc
    inference_library_version:  4.0.0b4
    sampling_time:              33.5627760887146
    tuning_steps:               1000

### 

In [172]:
test = alpha_results[:, :, 0] - alpha_results[:, :, 1]

In [175]:
alpha_results[:, :, 1:]

<xarray.DataArray 'α' (chain: 4, draw: 5000, α_dim_0: 3)>
array([[[-10.22478823,   3.27711342, -14.38744628],
        [-11.68876426,   4.47521427, -11.0741941 ],
        [-11.21865836,   4.71496246, -12.36258231],
        ...,
        [-11.4427235 ,   3.52808203, -11.7702213 ],
        [-12.5899542 ,   5.05628396, -12.67289458],
        [-11.95881544,   4.10630494, -11.86454411]],

       [[-12.95336371,   2.76646395, -11.01987314],
        [-11.27702268,   3.76339062, -14.4427701 ],
        [-10.8329693 ,   5.08478715, -12.24586618],
        ...,
        [-11.33828156,   3.34047249, -13.17797718],
        [-10.89015312,   2.5688046 , -13.78940017],
        [-10.89015312,   2.5688046 , -13.78940017]],

       [[-12.67092916,   2.98764234, -13.37849907],
        [-12.14141119,   3.72460297, -13.40563934],
        [-11.49368416,   3.8372382 , -10.29561026],
        ...,
        [-12.66732722,   5.11979102, -16.07472629],
        [-11.78988373,   4.92858634, -10.39376778],
        [-13.40079432,   5.2269072 , -13.50987368]],

       [[ -6.00694298,   0.48263666, -11.21056507],
        [-14.80768485,   4.46580174, -10.99661538],
        [-12.46459279,   3.99876703, -11.97188467],
        ...,
        [-11.45450366,   9.24366602, -13.54105802],
        [ -9.97379734,   2.01278552, -10.96910795],
        [-10.38526475,  -0.3181099 , -12.09544773]]])
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 4993 4994 4995 4996 4997 4998 4999
  * α_dim_0  (α_dim_0) int64 1 2 3

want:
1-2
1-3
1-4
2-3
2-4
3-4

In [184]:
    for name, (i, j) in zip(names, combos):
        pm.Deterministic(name, alpha[i] - alpha[j])

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


In [313]:
def get_diffs(alpha_results, names, combos):


In [315]:
get_diffs(alpha_results, names, combos)

         mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
α0-α1  31.433  2.965  26.075   36.866      0.019    0.014   26291.0   11517.0   

       r_hat  
α0-α1    1.0  
         mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
α0-α2  15.638  2.902  10.181   21.002      0.019    0.013   25038.0   10806.0   

       r_hat  
α0-α2    1.0  
         mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
α0-α3  32.289  2.877  26.842    37.61      0.018    0.013   26364.0   11180.0   

       r_hat  
α0-α3    1.0  
         mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
α1-α2 -15.795  2.907 -21.024  -10.133      0.019    0.014   24971.0    9201.0   

       r_hat  
α1-α2    1.0  
        mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
α1-α3  0.856  2.907  -4.486    6.105      0.019    0.032   25156.0    9594.0   

       r_hat  
α1-α3    1.0  
         mean     sd  hdi_3%  hdi_97%